In [1]:
# Exercise 1: Building a secure user authentication system
import bcrypt

class User:
    def __init__(self, username, password, priv_level = 'standard'):
        self.username = username
        self.password = self.hashed_password(password)
        self.priv_level = priv_level
        self.log_attempts = 0
        self.account_status = 'active'
        self.activity_log = []
        
    def hashed_password(self, password): # Creating the function to hash the password
        hashed_pw = password
        s = bcrypt.gensalt()
        h = bcrypt.hashpw(hashed_pw, s)
        return h
    
    def authenticate(self, password):
        if self.account_status == "locked":
            self.log_activity("Login attempt on locked account")
            return False
        if self.hashed_password(password)==self.password:
            self.log_attempts = 0
            self.log_activity("You're successfully logged in.")
            return True

        else:
            self.log_attempts += 1
            self.log_activity(f"Failed login attempts: {self.log_attempts}")
            if self.log_attempts >= 3:
                self.lock_account()
            return False
        
 
    def check_privileges(self, required_level):
        privilege_hierarchy = {'guest': 0, 'standard': 1, 'admin': 2}
        return privilege_hierarchy.get(self.__privilege_level, 0) >= privilege_hierarchy.get(required_level, 0)
   
    def lock_account():
        self.account_status = "locked"
        self.log_activity("Account is locked due to failed login attempts")

    def reset_login_attempts(self, admin_password):  # Because admin is the only person who can reset the password
        if self.hashed_password(admin_password) == "hashed_admin_secret":
            self.account_status = "active"
            self.log_attempts = 0
            self.log_activity("Account unblocked by admin")
            return True
        return False
        
    def log_activity(self, message):
        from datetime import datetime
        self.activity_log.append(f"{datetime.now()}: {message}")

    def get_safe_info(self):
        return {
            "username":self.username,
            "priv_level": self.priv_level,
            "account_status": self.account_status
        }

    def get_username(self):
        return self.username

    def get_priv_level(self):
        return self.priv_level
        
        

user_1 = (f"'Alice'", "1234", "standard")
user_2 = ("'Bob'", "fnr44", "guest")
user_3 = ("'Cait'", "DC43", "admin")




In [4]:
#Exercise 2: IoT Device Management System

from datetime import datetime, timedelta

class User:

    def __init__(self, user_id, role = "standard"):  
        self.user_id = user_id
        self.role = role
        self.is_admin = role == "admin" #Check if the user is admin

class Device:

    def __init__(self, device_id, device_type, owner, firmware_version = "v1.0"):
        self.device_id = device_id
        self.device_type = device_type
        self.owner = owner
        self.compliance_status = True
        self.last_security_scan = None
        self.is_active = True
        self.quarantined = False
        self.firmware_version = firmware_version

    def needs_scan(self):
        if not self.last_security_scan:
            return True #Never scanned
        return (datetime.now() - self.last_security_scan).days > 30 #Calculate days since last scan

    def run_security_scan(self): #Performs the scan and updates the compliance status
        self.last_security_scan = datetime.now()
        self.compliance_status = not self.needs_scan()
        return self.compliance_status

    def check_access(self, user, override = False):
        
        #Check the state of the device
        if not self.is_active:
            return False, "Device inactive"
        if self.quarantined:
            return False, "Device quarantined"

        is_owner = self.owner == user.user_id

        #Admin access
        if user.is_admin:
            if not self.compliance_status and not ovverride:
                return False, "Non compliant"
            elif override:
                return True, "Admin override"
            else:
                return True, "Admin access"

        #Standard user access
        else:
            if not is_owner:
                return False, "Not owner"
            if not self.compliance_status:
                return False, "Non compliant"
            return True, "Owner access"

    def quarantine(self):
        self.quarantined = True
        self.is_active = False


    def update_firmware(self, new_version, user):
        access, reason = self.check_access(user)
        if not access:
            return False, f"Cannot update firmware: {reason}"
        if self.quarantined:
            return False, "Cannot update firmware on quarantined device"
        if not new_version or new_version == self.firmware_version:
            return False, f"Invalid or same firmware version"

        #Update firmware
        old_version = self.firmware_version
        self.firmware_version = new_version

    def check_compliance(self):
        #Check device compliance and return status
        
        self.compliance_status = not self.needs_scan()  # Update compliance
        return {
            "device_id": self.device_id,
            "compliant": self.compliance_status,
            "last_scan": self.last_security_scan,
            "firmware_version": self.firmware_version,
            "needs_scan": self.needs_scan(),
            "quarantined": self.quarantined
        }
        
    #Get device info
    def get_info(self):
        status = "Compliant" if self.compliance_status else "Non-compliant"
        return f"Device: {self.device_id}, Type: {self.device_type}, Owner: {self.owner}, Status: {status}, Firmware: {self.firmware_version}"

class DeviceManager:
    def __init__(self):
        self.devices = {}
        self.logs = []

    def add(self, device):
        self.devices[device.device_id] = device
        self._log(f"Added {device.device_id}")

    def remove(self, device_id):
        if device_id in self.devices:
            del self.devices[device_id]
            self._log(f"Removed {device_id}")

            
    def authorise_access(self, device_id, user, override = False):
        if device_id not in self.devices:
            return False, "Device not found"

        if override and not user.is_admin:
            return False, "Only admin can override"

        device = self.devices[device_id]
        access, reason = device.check_access(user, override)

        self._log(f"{user.user_id} tried to access {device_id}: {access} - {reason}")
        return access, reason

    def quarantine(self, device_id, user):
        if not user.is_admin:
            return False, "Admin only"
        if device_id in self.devices:
            self.devices[device_id].quarantine()
            self._log(f"{user.user_id} quarantined {device_id}")
            return True, "Quarantined"
        return False, "Device not found"

     def update_firmware(self, device_id, new_version, user):
         #Update firmware for a spacific device
         if device_id not in self.devices:
             return False, "Device not found"
        device = self.devices[device_id]
        success, message = device.update_firmware(new_version, user)
        self._log(f"{user.user_id} attempted firmware update on {device_id}: {success} - {message}")

    def scan_all(self):
        for device in self.devices.values():
            if not device.quarantined:
                device.run_security_scan()

    def check_compliance_all(self):
        #Check compliance for all devices
        results = []
        for device in self.devices.values():
            results.append(device.check_compliance())
        return results

    def report(self):
        total = len(self.devices)
        compliant = sum(1 for d in self.devices.values() if d.compliant)
        quarantined = sum(1 for d in self.devices.values() if d.quarantined)

        return {
            "total": total,
            "compliant": compliant,
            "non_compliant": total - compliant,
            "quarantined": quarantined,
            "needs_scan": sum(1 for d in self.devices.values() if d.needs_scan))
        }

    def _log(self, message):
        self.logs.append(f"{datetime.now()}: {message}")
    
    def show_devices(self): #Print info about devices
        print(f"\nTotal devices: {len(self.devices)}")
        for device_id, device in self.devices.items():
            print(f"  {device.get_info()}")

if __name__ == "__main__":
    print("=== IoT Device Management System ===\n")
    
    admin = User("admin1", "admin")
    user1 = User("user1")
    user2 = User("user2")
    
    manager = DeviceManager()
    
    # Add devices with initial firmware versions
    dev1 = Device("temp001", "sensor", "user1", "v1.0")
    dev2 = Device("cam001", "camera", "user2", "v2.1")
    
    manager.add(dev1)
    manager.add(dev2)

    print("1. Initial device states:")
    manager.show_devices()
    
    print("\n2. Testing firmware updates:")
    
    # User tries to update their own device
    print("\n   a) User1 updates their own device:")
    success, msg = manager.update_firmware("temp001", "v1.5", user1)
    print(f"      Result: {success} - {msg}")
    
    # User tries to update someone else's device
    print("\n   b) User1 tries to update User2's device:")
    success, msg = manager.update_firmware("cam001", "v2.5", user1)
    print(f"      Result: {success} - {msg}")
    
    # Admin updates any device
    print("\n   c) Admin updates User2's device:")
    success, msg = manager.update_firmware("cam001", "v3.0", admin)
    print(f"      Result: {success} - {msg}")

    # Try to update quarantined device
    print("\n   d) Test update on quarantined device:")
    manager.quarantine("temp001", admin)  # Admin quarantines device
    success, msg = manager.update_firmware("temp001", "v2.0", user1)
    print(f"      Result: {success} - {msg}")
    
    # Try invalid update (same version)
    print("\n   e) Try invalid update (same version):")
    success, msg = manager.update_firmware("cam001", "v3.0", admin)
    print(f"      Result: {success} - {msg}")
    
    print("\n3. Testing compliance checks:")
    


IndentationError: unindent does not match any outer indentation level (<string>, line 134)

In [7]:
from datetime import datetime, timedelta

class User:
    def __init__(self, user_id, role="standard"):  
        self.user_id = user_id
        self.role = role
        self.is_admin = role == "admin"  # Check if the user is admin

class Device:
    def __init__(self, device_id, device_type, owner, firmware_version="v1.0"):
        self.device_id = device_id
        self.device_type = device_type
        self.owner = owner
        self.compliance_status = True
        self.last_security_scan = None
        self.is_active = True
        self.quarantined = False
        self.firmware_version = firmware_version

    def needs_scan(self):
        if not self.last_security_scan:
            return True  # Never scanned
        return (datetime.now() - self.last_security_scan).days > 30  # Calculate days since last scan

    def run_security_scan(self):  # Performs the scan and updates the compliance status
        self.last_security_scan = datetime.now()
        self.compliance_status = not self.needs_scan()
        return self.compliance_status

    def check_access(self, user, override=False):
        # Check the state of the device
        if not self.is_active:
            return False, "Device inactive"
        if self.quarantined:
            return False, "Device quarantined"

        is_owner = self.owner == user.user_id

        # Admin access
        if user.is_admin:
            if not self.compliance_status and not override:  # Fixed: was "ovverride"
                return False, "Non compliant"
            elif override:
                return True, "Admin override"
            else:
                return True, "Admin access"

        # Standard user access
        else:
            if not is_owner:
                return False, "Not owner"
            if not self.compliance_status:
                return False, "Non compliant"
            return True, "Owner access"  # Fixed: removed extra "True"

    def quarantine(self):
        self.quarantined = True
        self.is_active = False

    def update_firmware(self, new_version, user):
        access, reason = self.check_access(user)
        if not access:
            return False, f"Cannot update firmware: {reason}"
        if self.quarantined:
            return False, "Cannot update firmware on quarantined device"
        if not new_version or new_version == self.firmware_version:
            return False, f"Invalid or same firmware version"

        # Update firmware
        old_version = self.firmware_version
        self.firmware_version = new_version
        return True, f"Firmware updated from {old_version} to {new_version}" 

    def check_compliance(self):
        # Check device compliance and return status
        self.compliance_status = not self.needs_scan()  # Update compliance
        return {
            "device_id": self.device_id,
            "compliant": self.compliance_status,
            "last_scan": self.last_security_scan,
            "firmware_version": self.firmware_version,
            "needs_scan": self.needs_scan(),
            "quarantined": self.quarantined
        }
        
    # Get device info
    def get_info(self):
        status = "Compliant" if self.compliance_status else "Non-compliant"
        return f"Device: {self.device_id}, Type: {self.device_type}, Owner: {self.owner}, Status: {status}, Firmware: {self.firmware_version}"

class DeviceManager:
    def __init__(self):
        self.devices = {}
        self.logs = []

    def add(self, device):
        self.devices[device.device_id] = device
        self._log(f"Added {device.device_id}")

    def remove(self, device_id):
        if device_id in self.devices:
            del self.devices[device_id]
            self._log(f"Removed {device_id}")

    def authorise_access(self, device_id, user, override=False):
        if device_id not in self.devices:
            return False, "Device not found"

        if override and not user.is_admin:
            return False, "Only admin can override"

        device = self.devices[device_id]
        access, reason = device.check_access(user, override)

        self._log(f"{user.user_id} tried to access {device_id}: {access} - {reason}")
        return access, reason

    def quarantine(self, device_id, user):
        if not user.is_admin:
            return False, "Admin only"
        if device_id in self.devices:
            self.devices[device_id].quarantine()
            self._log(f"{user.user_id} quarantined {device_id}")
            return True, "Quarantined"
        return False, "Device not found"

    def update_firmware(self, device_id, new_version, user):
        # Update firmware for a specific device
        if device_id not in self.devices: 
            return False, "Device not found"
        device = self.devices[device_id]
        success, message = device.update_firmware(new_version, user)
        self._log(f"{user.user_id} attempted firmware update on {device_id}: {success} - {message}")
        return success, message

    def scan_all(self):
        for device in self.devices.values():
            if not device.quarantined:
                device.run_security_scan()

    def check_compliance_all(self):
        # Check compliance for all devices
        results = []
        for device in self.devices.values():
            results.append(device.check_compliance())
        return results

    def report(self):
        total = len(self.devices)
        compliant = sum(1 for d in self.devices.values() if d.compliance_status) 
        quarantined = sum(1 for d in self.devices.values() if d.quarantined)

        return {
            "total": total,
            "compliant": compliant,
            "non_compliant": total - compliant,
            "quarantined": quarantined,
            "needs_scan": sum(1 for d in self.devices.values() if d.needs_scan())  
        }

    def _log(self, message):
        self.logs.append(f"{datetime.now()}: {message}")
    
    def show_devices(self):  # Print info about devices
        print(f"\nTotal devices: {len(self.devices)}")
        for device_id, device in self.devices.items():
            print(f"  {device.get_info()}")

if __name__ == "__main__":
    print("=== IoT Device Management System ===\n")
    
    admin = User("admin1", "admin")
    user1 = User("user1")
    user2 = User("user2")
    
    manager = DeviceManager()
    
    #Add devices with initial firmware versions
    dev1 = Device("temp001", "sensor", "user1", "v1.0")
    dev2 = Device("cam001", "camera", "user2", "v2.1")
    
    manager.add(dev1)
    manager.add(dev2)

    print("1. Initial device states:")
    manager.show_devices()
    
    print("\n2. Testing firmware updates:")
    
    #User tries to update their own device
    print("\n   a) User1 updates their own device:")
    success, msg = manager.update_firmware("temp001", "v1.5", user1)
    print(f"      Result: {success} - {msg}")
    
    #User tries to update someone else's device
    print("\n   b) User1 tries to update User2's device:")
    success, msg = manager.update_firmware("cam001", "v2.5", user1)
    print(f"      Result: {success} - {msg}")
    
    #Admin updates any device
    print("\n   c) Admin updates User2's device:")
    success, msg = manager.update_firmware("cam001", "v3.0", admin)
    print(f"      Result: {success} - {msg}")

    #Try to update quarantined device
    print("\n   d) Test update on quarantined device:")
    manager.quarantine("temp001", admin)  # Admin quarantines device
    success, msg = manager.update_firmware("temp001", "v2.0", user1)
    print(f"      Result: {success} - {msg}")
    
    #Try invalid update (same version)
    print("\n   e) Try invalid update (same version):")
    success, msg = manager.update_firmware("cam001", "v3.0", admin)
    print(f"      Result: {success} - {msg}")
    
    print("\n3. Testing compliance checks:")

    #Check compliance
    compliance_status = dev1.check_compliance()
    print(f"\n   Device compliance status:")
    for key, value in compliance_status.items():
        print(f"      {key}: {value}")
 
    print("\n4. Recent logs:")
    for log in manager.logs[-5:]:
        print(f"   {log}")

=== IoT Device Management System ===

1. Initial device states:

Total devices: 2
  Device: temp001, Type: sensor, Owner: user1, Status: Compliant, Firmware: v1.0
  Device: cam001, Type: camera, Owner: user2, Status: Compliant, Firmware: v2.1

2. Testing firmware updates:

   a) User1 updates their own device:
      Result: True - Firmware updated from v1.0 to v1.5

   b) User1 tries to update User2's device:
      Result: False - Cannot update firmware: Not owner

   c) Admin updates User2's device:
      Result: True - Firmware updated from v2.1 to v3.0

   d) Test update on quarantined device:
      Result: False - Cannot update firmware: Device inactive

   e) Try invalid update (same version):
      Result: False - Invalid or same firmware version

3. Testing compliance checks:

   Device compliance status:
      device_id: temp001
      compliant: False
      last_scan: 2025-11-13 17:05:39.868695
      firmware_version: v1.5
      needs_scan: True
      quarantined: True

4. Rece